In [1]:
import numpy as np
import pandas as pd
import scipy as sc
import matplotlib.pyplot as plt
from math import *
from copy import *

def flatten(l):
    flat_list = [item for sublist in l for item in sublist]
    return flat_list

In [11]:
# Load the data

#lis_nyc1 = pd.read_csv('airbnb_similarity_insight/data/air-bnb-listings_nyc1.csv', sep = ';')
lis_nyc2 = pd.read_csv('airbnb_similarity_insight/data/airbnb-listings_nyc2.csv', sep = ';')
reviews = pd.read_csv('airbnb_similarity_insight/data/airbnb-reviews.csv', sep = ';')
rate_nyc = pd.read_csv('airbnb_similarity_insight/data/airbnb-ratings_nyc.csv', sep = ';')

/Users/bennett/anaconda3/envs/airbnb_p3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (83) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/bennett/anaconda3/envs/airbnb_p3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (67,85,87) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
# How many listings do I have total in nyc
   
unique_room_id_list = list(np.unique(lis_nyc2['ID']))
print(len(unique_room_id_list))


19528


In [ ]:
z

In [15]:
import copy
lis_nyc2_orig = deepcopy(lis_nyc2)
reviews_orig = deepcopy(lis_nyc2)


l2d = {'id':'room_id', 
        'geolocation':'coordinates', 
        'neighbourhood_cleansed': 'neighborhood_clean',
       'price':'room_price',
       'last_review':'date_last_review'}
lis_nyc2 = lis_nyc2.rename(columns=l2d)

lis_nyc2 = lis_nyc2.rename(columns={c:c.lower().replace(' ', '_') for c in lis_nyc2.columns})

reviews = reviews.rename(columns={c:c.lower().replace(' ', '_') for c in reviews.columns})

Index(['room_id', 'listing_url', 'scrape_id', 'last_scraped', 'name',
       'summary', 'space', 'description', 'experiences_offered',
       'neighborhood_overview', 'notes', 'transit', 'access', 'interaction',
       'house_rules', 'thumbnail_url', 'medium_url', 'picture_url',
       'xl_picture_url', 'host_id', 'host_url', 'host_name', 'host_since',
       'host_location', 'host_about', 'host_response_time',
       'host_response_rate', 'host_acceptance_rate', 'host_thumbnail_url',
       'host_picture_url', 'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications', 'street',
       'neighbourhood', 'neighborhood_clean', 'neighbourhood_group_cleansed',
       'city', 'state', 'zipcode', 'market', 'smart_location', 'country_code',
       'country', 'latitude', 'longitude', 'property_type', 'room_type',
       'accommodates', 'bathrooms', 'bedrooms', 'beds', 'bed_type',
       'amenities', 'square_feet', 'room_price', 'weekly_price',
       

In [16]:
# build a DataFrame of room_id --> host_name, list of reviewers, list of comments
# Takes some compute time (~10 minutes)
unique_room_id_list = sorted(unique_room_id_list)
hosts_for_room = {room_id: list(lis_nyc2[lis_nyc2.room_id == room_id].host_name) for room_id in unique_room_id_list}
reviewers_for_room = {room_id: list(reviews[reviews.listing_id == room_id].reviewer_id) for room_id in unique_room_id_list}
comments_for_room = {room_id: list(reviews[reviews.listing_id == room_id].comments) for room_id in unique_room_id_list}
room_db = pd.DataFrame({'room_id':unique_room_id_list,
                        'host_name':[hosts_for_room[k] for k in unique_room_id_list],
                       'user_id':[reviewers_for_room[k] for k in unique_room_id_list],
                       'comments': [comments_for_room[k] for k in unique_room_id_list]})

import pickle
with open('airbnb_similarity_insight/pickles/room_db_nyc.pkl', 'wb') as pickle_file:
    pickle.dump(room_db, pickle_file)

In [8]:
file = open("airbnb_similarity_insight/pickles/room_db_nyc.pkl",'rb')
object_file = pickle.load(file)
file.close()

EOFError: Ran out of input

In [17]:
# try some topic modeling
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2019)

In [18]:
# Tokenization: Split the text into sentences and the sentences into words. 
# Lowercase the words and remove punctuation.

# Words that have fewer than 3 characters are removed.

# Words are lemmatized — words in third person are changed to first person 
# and verbs in past and future tenses are changed into present.

# Words are stemmed — words are reduced to their root form.

# All stopwords are removed.

from nltk.stem import WordNetLemmatizer, SnowballStemmer
stemmer = SnowballStemmer("english")

def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

import nltk
nltk.download('wordnet')



[nltk_data] Downloading package wordnet to /Users/bennett/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [23]:
flat_doc_comments = []
hns = []
indices = []
rids = []
uids = []
c_and_i = list(zip(list(room_db.comments), 
                   list(room_db.host_name),
                   list(room_db.index), 
                   list(room_db.room_id),
                   list(room_db.user_id)))
for i, (cs, hn, ind, rid, uid) in enumerate(c_and_i):
    flat_doc_comments.extend(cs)
    hns.extend([hn]*len(cs))
    indices.extend([ind]*len(cs))
    rids.extend([rid]*len(cs))
    uids.extend([uid]*len(cs))

flat_room_db = pd.DataFrame({'listing_id':rids, 'host_name': hns, 'user_id':uids,
                            'comments':flat_doc_comments, 'indices':indices})

room_db.shape, flat_room_db.shape

((19528, 4), (325682, 5))

In [24]:
#data_text = room_db[['comments']]
#data_text['index'] = data_text.index
#bumpy_documents = data_text
#doc_sample = bumpy_documents[bumpy_documents['index'] == 4310].values[0][0]

doc_sample = flat_room_db.comments[122]
print(doc_sample)

print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')

print(preprocess(doc_sample))

An unforgettable trip to NYC. Fantastic city with crazy people an much to see.
original document: 
['An', 'unforgettable', 'trip', 'to', 'NYC.', 'Fantastic', 'city', 'with', 'crazy', 'people', 'an', 'much', 'to', 'see.']


 tokenized and lemmatized document: 
['unforgett', 'trip', 'fantast', 'citi', 'crazi', 'peopl']


In [25]:
from langdetect import detect
detect("War doesn't show who's right, just who's left.")

'en'

In [34]:
flat_room_db.comments = flat_room_db.comments.fillna('')
for ic, c in enumerate(flat_room_db.comments):
    name = flat_room_db.host_name[ic][0]
    if name is not np.nan:
        flat_room_db.comments[ic]=flat_room_db.comments[ic].replace(name, 'HOST_NAME')

/Users/bennett/anaconda3/envs/airbnb_p3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


KeyboardInterrupt: 

In [39]:
test = flat_room_db[['host_name', 'comments']]
#df['d'] = df.apply(lambda x: some_func(a = x['a'], b = x['b'], c = x['c']), axis=1)
def clean_name_df(x):
    if type(x['host_name'][0]) is str:
        return x['comments'].replace(x['host_name'][0], 'HOST_NAME')
    else:
        return x['comments']
test.apply(lambda x: clean_name_df(x), axis=1)

0         Stephanie's offered all the most important thi...
1                        awesome couldn't have been better.
2         Nous avons passé un agréable séjour chez Stéph...
3         We stayed at the 111th Street apartment with S...
4         Very conveniently located just North of Centra...
                                ...                        
325677                  Great stay, great place, great host
325678    Great experience! Apartment was clean and tidy...
325679    margarita is the perfect host. we very much en...
325680    HOST_NAME is a great person, the place below t...
325681    HOST_NAME was a great host! The linens were cl...
Length: 325682, dtype: object

In [41]:
# ONLY LOOKS AT REVIEWS WITH 3+ CHARACTERS
# ONLY LOOK AT REVIEWS IN ENGLISH

def clean_name_df(x):
    if type(x['host_name'][0]) is str:
        return x['comments'].replace(x['host_name'][0], 'HOST_NAME')
    else:
        return x['comments']
flat_room_db['comments'] = flat_room_db.apply(lambda x: clean_name_df(x), axis=1)

flat_room_db = flat_room_db[flat_room_db.comments.apply(lambda x: len(x) >= 3)]

def clean_detect(x):
    try:
        return detect(x)
    except:
        return 'No Language'



NameError: name 'flatt_room_db' is not defined

In [42]:
flat_room_db = flat_room_db[flat_room_db.comments.apply(lambda x: clean_detect(x) =='en')]

In [78]:
flat_room_db.comments.apply(lambda x: x[:34] != 'The host canceled this reservation')[12]

False

In [80]:
print(flat_room_db.comments)
# Remove comments with automated message

flat_room_db = flat_room_db[flat_room_db.comments.apply(lambda x: x[:34] != 'The host canceled this reservation')]
             
print(flat_room_db.comments)

0         Stephanie's offered all the most important thi...
1                        awesome couldn't have been better.
3         We stayed at the 111th Street apartment with S...
4         Very conveniently located just North of Centra...
5         We had a great stay with Stephanie and her fam...
                                ...                        
325677                  Great stay, great place, great host
325678    Great experience! Apartment was clean and tidy...
325679    margarita is the perfect host. we very much en...
325680    HOST_NAME is a great person, the place below t...
325681    HOST_NAME was a great host! The linens were cl...
Name: comments, Length: 280476, dtype: object
0         Stephanie's offered all the most important thi...
1                        awesome couldn't have been better.
3         We stayed at the 111th Street apartment with S...
4         Very conveniently located just North of Centra...
5         We had a great stay with Stephanie and her f

In [85]:
with open('airbnb_similarity_insight/pickles/flat_room_db_nyc_clean.pkl', 'wb') as pickle_file:
    pickle.dump(flat_room_db, pickle_file)

In [84]:
print(len(flat_room_db.comments))
processed_comments = flat_room_db['comments'].map(preprocess)

280476


In [86]:
# Create a dictionary from ‘processed_docs’ containing the number of times a word appears in the training set.
dictionary = gensim.corpora.Dictionary(processed_comments)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 50:
        break

0 access
1 away
2 blanket
3 clean
4 comfort
5 easi
6 feel
7 find
8 fresh
9 futur
10 generous
11 home
12 import
13 invit
14 manhattan
15 offer
16 open
17 quiet
18 room
19 stephani
20 thing
21 towel
22 travel
23 warm
24 welcom
25 awesom
26 better
27 couldn
28 abl
29 advis
30 altern
31 apart
32 card
33 ensur
34 famili
35 furthest
36 hotel
37 long
38 matter
39 overpric
40 place
41 recommend
42 stay
43 street
44 time
45 tourist
46 transport
47 unlimit
48 uptown
49 want
50 york


In [87]:
# Filter out tokens that appear in
#less than no_below documents (absolute number) or
#more than no_above documents (fraction of total corpus size, not absolute number).
#after the above two steps, keep only the first 100000 most frequent tokens.

dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=100000)

#Gensim doc2bow
#For each document we create a dictionary reporting how many
#words and how many times those words appear. Save this to ‘bow_corpus’, then check our selected document earlier.
bow_corpus = [dictionary.doc2bow(comment) for comment in processed_comments]
bow_corpus[4310]

# Preview Bag Of Words for our sample preprocessed document.
bow_doc_4310 = bow_corpus[4310]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                               dictionary[bow_doc_4310[i][0]], 
                                                bow_doc_4310[i][1]))

Word 40 ("place") appears 1 time.
Word 41 ("recommend") appears 1 time.
Word 65 ("locat") appears 1 time.
Word 140 ("perfect") appears 1 time.
Word 202 ("definit") appears 1 time.
Word 208 ("love") appears 1 time.
Word 1188 ("vibe") appears 1 time.
Word 1251 ("cool") appears 1 time.


In [88]:
#Create tf-idf model object using models.TfidfModel on ‘bow_corpus’ and save it to ‘tfidf’, 
# then apply transformation to the entire corpus and call it ‘corpus_tfidf’.
# Finally we preview TF-IDF scores for our first document.
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.12471894765745287),
 (1, 0.10890457556812892),
 (2, 0.23639534226884684),
 (3, 0.04648951036054526),
 (4, 0.13275091292456642),
 (5, 0.07982773410548202),
 (6, 0.10822668850791249),
 (7, 0.2416419288644195),
 (8, 0.19115722972215154),
 (9, 0.1798599738225504),
 (10, 0.21316894590729127),
 (11, 0.3009188482047445),
 (12, 0.19454932992954152),
 (13, 0.22285834989627018),
 (14, 0.11837266494933073),
 (15, 0.158173881354976),
 (16, 0.1663581869207859),
 (17, 0.10037984563225434),
 (18, 0.06951682790986437),
 (19, 0.6013990840134688),
 (20, 0.12570547642032107),
 (21, 0.1484860647727601),
 (22, 0.13629457837964543),
 (23, 0.13835259755623772),
 (24, 0.0958091666977137)]


In [90]:
# Running LDA using Bag of Words
# Train our lda model using gensim.models.LdaMulticore and save it to ‘lda_model’
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=20, id2word=dictionary, passes=2, workers=5)

In [91]:
# For each topic, we will explore the words occuring in that topic and its relative weight.

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.026*"apart" + 0.022*"kitchen" + 0.017*"towel" + 0.015*"need" + 0.014*"provid" + 0.010*"clean" + 0.009*"comfort" + 0.008*"bedroom" + 0.008*"coffe" + 0.007*"cook"
Topic: 1 
Words: 0.046*"park" + 0.045*"central" + 0.038*"apart" + 0.036*"locat" + 0.036*"walk" + 0.034*"squar" + 0.031*"time" + 0.024*"subway" + 0.021*"close" + 0.020*"block"
Topic: 2 
Words: 0.040*"space" + 0.027*"room" + 0.019*"comfort" + 0.016*"love" + 0.014*"bedroom" + 0.012*"bathroom" + 0.012*"apart" + 0.012*"privat" + 0.012*"host" + 0.011*"live"
Topic: 3 
Words: 0.042*"apart" + 0.023*"nois" + 0.020*"locat" + 0.019*"night" + 0.018*"sleep" + 0.016*"street" + 0.013*"clean" + 0.012*"place" + 0.011*"noisi" + 0.010*"comfort"
Topic: 4 
Words: 0.040*"apart" + 0.038*"locat" + 0.033*"perfect" + 0.029*"host" + 0.028*"love" + 0.025*"nice" + 0.025*"thank" + 0.024*"help" + 0.018*"place" + 0.017*"question"
Topic: 5 
Words: 0.067*"locat" + 0.061*"apart" + 0.043*"host" + 0.042*"clean" + 0.035*"place" + 0.033*"recommend"

In [92]:
# Running LDA using TF-IDF

lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=20, id2word=dictionary, passes=2, workers=5)

for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.894444465637207	 
Topic: 0.033*"place" + 0.030*"nice" + 0.029*"host" + 0.028*"locat" + 0.022*"good" + 0.021*"clean" + 0.019*"perfect" + 0.019*"recommend" + 0.016*"apart" + 0.016*"thank"


In [93]:
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.013*"cute" + 0.008*"nice" + 0.008*"place" + 0.007*"apart" + 0.007*"friend" + 0.007*"room" + 0.006*"cat" + 0.006*"host" + 0.006*"clean" + 0.006*"help"
Topic: 1 
Words: 0.019*"villag" + 0.016*"east" + 0.009*"apart" + 0.009*"west" + 0.009*"soho" + 0.009*"place" + 0.008*"restaur" + 0.008*"heart" + 0.008*"locat" + 0.007*"walk"
Topic: 2 
Words: 0.006*"place" + 0.006*"apart" + 0.006*"hide" + 0.005*"free" + 0.005*"time" + 0.005*"easi" + 0.004*"clean" + 0.004*"sensit" + 0.004*"nice" + 0.004*"room"
Topic: 3 
Words: 0.007*"room" + 0.006*"apart" + 0.005*"place" + 0.005*"floor" + 0.005*"good" + 0.005*"night" + 0.005*"bathroom" + 0.005*"nice" + 0.005*"time" + 0.004*"clean"
Topic: 4 
Words: 0.015*"central" + 0.015*"park" + 0.013*"walk" + 0.012*"close" + 0.012*"subway" + 0.011*"restaur" + 0.010*"place" + 0.010*"squar" + 0.010*"apart" + 0.009*"locat"
Topic: 5 
Words: 0.007*"room" + 0.006*"apart" + 0.006*"good" + 0.006*"night" + 0.006*"nois" + 0.005*"sleep" + 0.005*"bedroom" + 0.005*"

In [94]:
# Performance evaluation by classifying sample document using LDA TF-IDF model.
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.894444465637207	 
Topic: 0.033*"place" + 0.030*"nice" + 0.029*"host" + 0.028*"locat" + 0.022*"good" + 0.021*"clean" + 0.019*"perfect" + 0.019*"recommend" + 0.016*"apart" + 0.016*"thank"
